In [1]:
from PIL import Image
from os.path import join
import imageio
from torch import nn
from torch.nn.modules.linear import Linear
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import os,sys,os.path
import pandas as pd
import pickle
import pydicom
import glob
import collections
import pprint
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms.functional as TF
import warnings
import tarfile
import zipfile
import random
from skimage.io import imread, imsave
from skimage.transform import resize
import hashlib
import pickle
from tqdm import tqdm
import time
import math
from torch.utils.data import Dataset
import argparse
from sklearn.cluster import KMeans
import scipy
import torch.nn as nn
import torch.optim as optim
import copy
from scipy.optimize import linear_sum_assignment as linear_assignment
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorboardX import SummaryWriter

In [3]:
from option import args_parser
from models import CIFAR_VAE
from utils import save_decoded_image,Accuracy
from sampling import Dataset_IIDsampling, LocalDataset, LocalDataloaders, average_weights, Dataset_non_IIDsampling, get_dataset_cifar10_extr_noniid,partition_data,LocalDataloaders_sample
from Localupdate_VAE import LocalUpdate

In [4]:
torch.set_default_dtype(torch.float64)
print(torch.__version__)
torch.cuda.is_available()
device = torch.device("cuda:0")
print(device)
args = args_parser()
np.set_printoptions(threshold=np.inf)
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5,6,7'

1.9.0+rocm4.0.1
cuda:0


In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

Files already downloaded and verified


In [7]:
args = args_parser()
args.tag = 'federated'

In [8]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
args.num_users = 10
# trainset, testset, dict_users, dict_users_test = get_dataset_cifar10_extr_noniid(args.num_users, 5, 200, 1.0, rand_seed = 10,all_class = False, all_class_prop = 0.05)
train_dataset,testset, dict_users, traindata_cls_counts = partition_data(n_users = args.num_users, alpha=0.5,rand_seed = 0)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
Loaders_train = LocalDataloaders_sample(train_dataset,dict_users,args.batch_size,ShuffleorNot = True, mini=0.4)
Major_classes = []
Total_classes = []
for idx in range(args.num_users):
    counts = [0,0,0,0,0,0,0,0,0,0]
    for batch_idx,(X,y) in enumerate(Loaders_train[idx]):
        batch = len(y)
        y = np.array(y)
        for i in range(batch):
            counts[int(y[i])] += 1
    counts = np.array(counts)
    print(counts)
    major_classes = counts.argsort()[::-1][0:3]
    Total_classes.append(counts/np.sum(counts))
    Major_classes.append(major_classes)

[ 354   58   72   18  273  344    0 1003    0    0]
[  8  11 378 647  35  78   0 274  43   1]
[  4   2 111 313 341 118  47 429 383  26]
[173 476  15  27 346 260  38   7 550 126]
[ 206 1180  243  276   33  137    0    0    0    0]
[500  40 115  50 620   5   0  35 156   0]
[  2 115 669  14   0  52 756   7 465   0]
[288   2 119 302   9 375 252   0  28 355]
[ 451   60  285  180  306  460   39    6    0 1444]
[  1  79  35 121  45 172 849 259 366  43]


In [11]:
from scipy.stats import wasserstein_distance
L2D = np.zeros((args.num_users,args.num_users))
for i in range(args.num_users):
    for j in range(args.num_users):
        L2D[i,j] = np.linalg.norm(Total_classes[i]-Total_classes[j])

In [12]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [13]:
global_model = CIFAR_VAE(in_channels=3,input_shape=[32,32],code_length= 128, classes = 10,drop_rate=0.2,decoder = True)

In [14]:
print('# model parameters:', sum(param.numel() for param in global_model.parameters()))

# model parameters: 1840340


In [15]:
# use multi-GPU to train parallelly
global_model = nn.DataParallel(global_model)
global_model.to(device)
for m in global_model.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))

In [16]:
optimizer = optim.Adam(global_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.5)

In [17]:
logger = SummaryWriter('../logs')
checkpoint_dir = './checkpoint/'+args.tag+'/VAE_CIFAR/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
with open(checkpoint_dir+'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)
print('Data and model loaded')
print('Checkpoint dir:', checkpoint_dir)

Data and model loaded
Checkpoint dir: ./checkpoint/federated/VAE_CIFAR/


In [18]:
# copy weights
global_weights = global_model.state_dict()

In [21]:
# training
args.num_epochs = 50
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
for epoch in tqdm(range(args.num_epochs)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    global_model.train()
    m = max(int(args.update_frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in idxs_users:
        local_model = LocalUpdate(args,Loaders_train[idx], idxs=dict_users[idx], logger=logger)
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch, u=0.1, device=device)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))


    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)
    print("average loss:  ", loss_avg)

  0%|          | 0/50 [00:00<?, ?it/s]/home/huancheng/miniconda3/envs/anomaly/lib/python3.6/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/1970 (0%)]KLD: 2.067118 BCE: 2045.460053 Acc: 0.765625
| Global Round : 0 | Local Epoch : 0 | [640/1970 (32%)]KLD: 1.942955 BCE: 2050.066800 Acc: 0.843750
| Global Round : 0 | Local Epoch : 0 | [1280/1970 (65%)]KLD: 2.047074 BCE: 2044.592783 Acc: 0.906250
| Global Round : 0 | Local Epoch : 0 | [1500/1970 (97%)]KLD: 2.007803 BCE: 2038.466746 Acc: 0.920000
| Global Round : 0 | Local Epoch : 1 | [0/1970 (0%)]KLD: 1.938756 BCE: 2048.254635 Acc: 0.968750
| Global Round : 0 | Local Epoch : 1 | [640/1970 (32%)]KLD: 2.077756 BCE: 2052.521852 Acc: 0.953125
| Global Round : 0 | Local Epoch : 1 | [1280/1970 (65%)]KLD: 2.032891 BCE: 2056.000518 Acc: 0.968750
| Global Round : 0 | Local Epoch : 1 | [1500/1970 (97%)]KLD: 2.125314 BCE: 2050.247671 Acc: 0.960000
| Global Round : 0 | Local Epoch : 2 | [0/1970 (0%)]KLD: 2.097002 BCE: 2040.398082 Acc: 0.968750
| Global Round : 0 | Local Epoch : 2 | [640/1970 (32%)]KLD: 2.045725 BCE

  2%|▏         | 1/50 [01:46<1:26:37, 106.08s/it]

| Global Round : 0 | Local Epoch : 4 | [1920/2018 (94%)]KLD: 2.018601 BCE: 2055.761000 Acc: 1.000000
average loss:   205.05691973807689

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/1970 (0%)]KLD: 2.031797 BCE: 2046.965033 Acc: 0.765625
| Global Round : 1 | Local Epoch : 0 | [640/1970 (32%)]KLD: 1.958714 BCE: 2054.718546 Acc: 0.828125
| Global Round : 1 | Local Epoch : 0 | [1280/1970 (65%)]KLD: 1.965351 BCE: 2045.958882 Acc: 0.875000
| Global Round : 1 | Local Epoch : 0 | [1500/1970 (97%)]KLD: 2.016170 BCE: 2058.650647 Acc: 0.920000
| Global Round : 1 | Local Epoch : 1 | [0/1970 (0%)]KLD: 2.035477 BCE: 2042.236605 Acc: 0.984375
| Global Round : 1 | Local Epoch : 1 | [640/1970 (32%)]KLD: 2.019070 BCE: 2050.577217 Acc: 0.937500
| Global Round : 1 | Local Epoch : 1 | [1280/1970 (65%)]KLD: 2.011448 BCE: 2056.421385 Acc: 0.937500
| Global Round : 1 | Local Epoch : 1 | [1500/1970 (97%)]KLD: 2.084690 BCE: 2051.856291 Acc: 0.980000
| Global Round : 1 | Local Epoch 

  4%|▍         | 2/50 [03:31<1:24:39, 105.83s/it]

...

average loss:   204.89805276213443

 | Global Training Round : 49 |

| Global Round : 48 | Local Epoch : 0 | [0/1730 (0%)]KLD: 1.999950 BCE: 2051.997493 Acc: 0.890625
| Global Round : 48 | Local Epoch : 0 | [640/1730 (36%)]KLD: 1.991633 BCE: 2044.849990 Acc: 0.875000
| Global Round : 48 | Local Epoch : 0 | [1280/1730 (71%)]KLD: 2.045668 BCE: 2030.036379 Acc: 0.906250
| Global Round : 48 | Local Epoch : 1 | [0/1730 (0%)]KLD: 2.002858 BCE: 2050.531414 Acc: 0.984375
| Global Round : 48 | Local Epoch : 1 | [640/1730 (36%)]KLD: 2.211626 BCE: 2038.708608 Acc: 0.921875
| Global Round : 48 | Local Epoch : 1 | [1280/1730 (71%)]KLD: 1.993164 BCE: 2038.748067 Acc: 0.953125
| Global Round : 48 | Local Epoch : 2 | [0/1730 (0%)]KLD: 1.933323 BCE: 2052.345457 Acc: 1.000000
| Global Round : 48 | Local Epoch : 2 | [640/1730 (36%)]KLD: 2.041213 BCE: 2049.759444 Acc: 0.937500
| Global Round : 48 | Local Epoch : 2 | [1280/1730 (71%)]KLD: 2.127913 BCE: 2031.125444 Acc: 0.953125
| Global Round : 48 | Local 

 98%|█████████▊| 49/50 [1:26:07<01:45, 105.31s/it]

average loss:   204.91952437095284

 | Global Training Round : 50 |

| Global Round : 49 | Local Epoch : 0 | [0/2075 (0%)]KLD: 2.018575 BCE: 2044.357593 Acc: 0.812500
| Global Round : 49 | Local Epoch : 0 | [640/2075 (30%)]KLD: 2.132081 BCE: 2033.821215 Acc: 0.890625
| Global Round : 49 | Local Epoch : 0 | [1280/2075 (61%)]KLD: 2.006491 BCE: 2048.356672 Acc: 0.968750
| Global Round : 49 | Local Epoch : 0 | [1920/2075 (91%)]KLD: 1.991030 BCE: 2029.855992 Acc: 0.953125
| Global Round : 49 | Local Epoch : 1 | [0/2075 (0%)]KLD: 2.038226 BCE: 2034.032560 Acc: 0.968750
| Global Round : 49 | Local Epoch : 1 | [640/2075 (30%)]KLD: 2.074924 BCE: 2036.928823 Acc: 0.984375
| Global Round : 49 | Local Epoch : 1 | [1280/2075 (61%)]KLD: 2.104879 BCE: 2035.570084 Acc: 0.968750
| Global Round : 49 | Local Epoch : 1 | [1920/2075 (91%)]KLD: 2.098164 BCE: 2028.984720 Acc: 1.000000
| Global Round : 49 | Local Epoch : 2 | [0/2075 (0%)]KLD: 2.056506 BCE: 2031.432371 Acc: 1.000000
| Global Round : 49 | Local

100%|██████████| 50/50 [1:27:52<00:00, 105.45s/it]

average loss:   204.8963819565983


In [23]:
loader_test = torch.utils.data.DataLoader(testset, batch_size=128,shuffle=False, num_workers=2)
accuracy = 0
count = 0
global_model.eval()
for cnt, (X,y) in enumerate(loader_test):
    X = X.to(device)
    m = torch.nn.Sigmoid()
    X = m(X)
    y = y.double().to(device)
    mu,logVar,p,_X = global_model(X)
    y_pred = p.argmax(1)
    accuracy += Accuracy(y,y_pred)
    count += 1
print("accuracy of test:",accuracy/(count-1))

accuracy of test: 0.6685697115384616
